In [2]:
from nba_api.stats.static import teams
import pandas as pd
import time
pd.set_option('display.max_columns', None) 

In [3]:
# Get all Team IDs
team_dict = teams.get_teams()
id_team_pairs = [(team['id'], team['full_name']) for team in team_dict]
team_ids = [pair[0] for pair in id_team_pairs]
#team_ids

In [25]:
# Get all team Stats
from nba_api.stats.endpoints import leaguegamefinder

all_teams_data_frames = []

# Loop through each team ID and fetch the corresponding team stats
for team_id in team_ids:
    team_games = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_id)
    time.sleep(.600)
    team_data_frame = team_games.get_data_frames()[0]
    # team_data_frame = team_data_frame[team_data_frame['SEASON_ID'].str[1:] >= "2021"]
    all_teams_data_frames.append(team_data_frame)

In [26]:
all_teams_data_frames[0]

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22023,1610612737,ATL,Atlanta Hawks,0022300521,2024-01-10,ATL vs. PHI,W,265,139,48,102,0.471,12,34,0.353,31,42,0.738,16,39,55,32,6.0,8,12,21,7.0
1,22023,1610612737,ATL,Atlanta Hawks,0022300499,2024-01-07,ATL @ ORL,L,265,110,42,91,0.462,9,30,0.300,17,29,0.586,8,42,50,21,9.0,10,12,23,-7.0
2,22023,1610612737,ATL,Atlanta Hawks,0022300480,2024-01-05,ATL @ IND,L,238,116,40,93,0.430,10,42,0.238,26,30,0.867,14,25,39,21,6.0,3,14,11,-34.0
3,22023,1610612737,ATL,Atlanta Hawks,0022300467,2024-01-03,ATL vs. OKC,W,240,141,48,95,0.505,14,39,0.359,31,33,0.939,15,34,49,30,8.0,5,11,21,3.0
4,22023,1610612737,ATL,Atlanta Hawks,0022300445,2023-12-31,ATL @ WAS,W,240,130,47,95,0.495,12,32,0.375,24,37,0.649,12,46,58,27,5.0,3,12,18,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3644,21983,1610612737,ATL,Atlanta Hawks,0028300058,1983-11-06,ATL @ MIL,L,240,84,32,78,0.410,0,2,0.000,20,30,0.667,11,26,37,18,4.0,5,14,24,NaN
3645,21983,1610612737,ATL,Atlanta Hawks,0028300041,1983-11-04,ATL vs. CHI,W,240,103,42,86,0.488,1,1,1.000,18,26,0.692,19,27,46,31,14.0,13,18,27,NaN
3646,21983,1610612737,ATL,Atlanta Hawks,0028300027,1983-11-01,ATL vs. WAS,W,240,95,38,81,0.469,0,0,NaN,19,30,0.633,12,29,41,20,7.0,10,16,34,NaN
3647,21983,1610612737,ATL,Atlanta Hawks,0028300014,1983-10-29,ATL vs. DET,W,240,117,49,94,0.521,0,1,0.000,19,30,0.633,27,21,48,28,14.0,7,23,35,NaN


In [4]:
# Get all Player ID
from nba_api.stats.static import players

player_dict = players.get_players()
id_player_pairs = [(player['id'], player['full_name']) for player in player_dict]
player_ids = [pair[0] for pair in id_player_pairs]
#player_ids

In [15]:
from concurrent.futures import ThreadPoolExecutor
from nba_api.stats.endpoints import playergamelog
from nba_api.stats.library.parameters import SeasonAll
from retrying import retry

@retry(stop_max_attempt_number=3, wait_fixed=2000)

def fetch_player_data(player_id):
    player_gamelog = playergamelog.PlayerGameLog(player_id, season=SeasonAll.all)
    time.sleep(.600)
    return player_gamelog.get_data_frames()[0]

with ThreadPoolExecutor() as executor:
    all_players_data_frames = list(executor.map(fetch_player_data, player_ids))

ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

In [7]:
from nba_api.stats.endpoints import playergamelog
from nba_api.stats.library.parameters import SeasonAll
# Loop through each player ID and fetch the corresponding team stats
all_players_data_frames = []
for player_id in player_ids:
    player_gamelog = playergamelog.PlayerGameLog(player_id, season = SeasonAll.all)
    time.sleep(.600)
    player_gamelog_df = player_gamelog.get_data_frames()[0]
    all_players_data_frames.append(player_gamelog_df)
    print(len(all_players_data_frames))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104


KeyboardInterrupt: 

In [5]:
from concurrent.futures import ThreadPoolExecutor
from nba_api.stats.endpoints import playergamelog
from nba_api.stats.library.parameters import SeasonAll
import concurrent, time

# Function to fetch player game logs for a given player ID
def fetch_player_gamelog(player_id):
    player_gamelog = playergamelog.PlayerGameLog(player_id, season=SeasonAll.all)
    player_gamelog_df = player_gamelog.get_data_frames()[0]
    return player_gamelog_df

# Using ThreadPoolExecutor for concurrent execution
all_players_data_frames = []
with ThreadPoolExecutor() as executor:
    # Fetch player game logs concurrently
    futures = [executor.submit(fetch_player_gamelog, player_id) for player_id in player_ids]

    # Collect the results as they become available
    for future in concurrent.futures.as_completed(futures):
        player_gamelog_df = future.result()
        all_players_data_frames.append(player_gamelog_df)
        time.sleep(.600)
        print(len(all_players_data_frames))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172


KeyboardInterrupt: 